<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/TESLA_NEMO_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!uv pip install "nvidia-nat[all]"

In [4]:
!nat --version

nat, version 1.3.1


In [ ]:
#meta/llama-3.1-405b-instruct

In [38]:
# TESLA_NEMO_DEMO_FINAL_PRODUCTION.ipynb
# Production-ready Tesla Swarm Energy Coordinator with 405B model

import os
import asyncio
import yaml
import re
import warnings
from datetime import datetime
from google.colab import userdata
from nat.runtime.loader import load_workflow

# ============================================================================
# CONFIGURATION & INITIALIZATION
# ============================================================================

# Suppress warnings for clean output
warnings.filterwarnings('ignore')
import logging
logging.getLogger('nat.profiler').setLevel(logging.ERROR)

# Set API Key
os.environ["NVIDIA_API_KEY"] = userdata.get('NVIDIA_API_KEY')

# Production Configuration
PRODUCTION_CONFIG = {
    "llms": {
        "tesla_llm": {
            "_type": "nim",
            "model_name": "meta/llama-3.1-405b-instruct",
            "api_key": "${NVIDIA_API_KEY}",
            "temperature": 0.1,
            "max_tokens": 200,
            "top_p": 0.9,
            "frequency_penalty": 0.2
        }
    },
    "workflow": {
        "_type": "chat_completion",
        "llm_name": "tesla_llm",
        "prompt": "Analyze drone report for SOC (State of Charge): {task}"
    }
}

# Color codes for terminal output
TERMINAL_COLORS = {
    "header": "\033[94m",
    "success": "\033[92m",
    "warning": "\033[93m",
    "critical": "\033[91m",
    "info": "\033[96m",
    "reset": "\033[0m"
}

# ============================================================================
# CORE BUSINESS LOGIC
# ============================================================================

class TeslaSwarmCoordinator:
    """Tesla Perpetual Swarm Energy Coordinator"""

    # Business rules
    TRANSFER_THRESHOLD = 0.85
    CRITICAL_THRESHOLD = 0.20
    HIGH_THRESHOLD = 0.50

    @staticmethod
    def extract_soc_from_report(report_text):
        """Extract SOC value from drone report text"""
        # Pattern to match SOC values in various formats
        soc_patterns = [
            r'SOC\s+(\d+\.\d+)',           # SOC 0.12
            r'SOC[:=]\s*(\d+\.\d+)',       # SOC: 0.12, SOC=0.12
            r'State of Charge\s*[:=]?\s*(\d+\.\d+)',  # State of Charge 0.12
            r'(\d+\.\d+)\s+SOC',           # 0.12 SOC
            r'battery\s*[:=]\s*(\d+\.\d+)' # battery: 0.12
        ]

        for pattern in soc_patterns:
            match = re.search(pattern, report_text, re.IGNORECASE)
            if match:
                try:
                    soc = float(match.group(1))
                    # Validate it's a reasonable SOC value
                    if 0.0 <= soc <= 1.0:
                        return soc
                except (ValueError, TypeError):
                    continue

        return None

    @staticmethod
    def is_faulty_report(report_text):
        """Check if report indicates faulty/unreadable SOC"""
        faulty_indicators = [
            'unknown', 'error', 'fault', 'fail',
            'unreadable', 'n/a', '--', 'not available'
        ]
        return any(indicator in report_text.lower()
                  for indicator in faulty_indicators)

    @staticmethod
    def determine_transfer_priority(soc):
        """Determine priority level for power transfer"""
        if soc < TeslaSwarmCoordinator.CRITICAL_THRESHOLD:
            return ("🔴 CRITICAL", "Immediate emergency transfer required")
        elif soc < TeslaSwarmCoordinator.HIGH_THRESHOLD:
            return ("🟡 HIGH", "High priority transfer needed")
        else:
            return ("🟢 STANDARD", "Standard priority transfer")

    @staticmethod
    def determine_charge_level(soc):
        """Determine descriptive charge level"""
        if soc > 0.95:
            return "FULL", f"{soc*100:.0f}%"
        elif soc > 0.85:
            return "HIGH", f"{soc*100:.0f}%"
        elif soc > 0.50:
            return "ADEQUATE", f"{soc*100:.0f}%"
        elif soc > 0.20:
            return "LOW", f"{soc*100:.0f}%"
        else:
            return "CRITICAL", f"{soc*100:.0f}%"

    @staticmethod
    def generate_coordinator_output(drone_name, soc, position, is_faulty=False):
        """Generate formatted coordinator decision output"""

        if is_faulty or soc is None:
            return {
                "status": "FAULTY",
                "reasoning": f"SOC data unavailable or unreadable for {drone_name}",
                "decision": "MANUAL REVIEW REQUIRED",
                "details": f"Dispatch inspection team to {drone_name} at position {position}",
                "icon": "🔍",
                "color": TERMINAL_COLORS["info"]
            }

        # Apply business rules
        if soc <= TeslaSwarmCoordinator.TRANSFER_THRESHOLD:
            comparison = "below" if soc < TeslaSwarmCoordinator.TRANSFER_THRESHOLD else "equal to"
            priority, priority_desc = TeslaSwarmCoordinator.determine_transfer_priority(soc)

            return {
                "status": "TRANSFER_REQUIRED",
                "reasoning": f"SOC {soc:.2f} is {comparison} the {TeslaSwarmCoordinator.TRANSFER_THRESHOLD} threshold",
                "decision": "TRANSFER REQUIRED",
                "details": f"Initiate {priority} priority power transfer to {drone_name}. {priority_desc}",
                "icon": "⚡",
                "color": TERMINAL_COLORS["warning"] if soc >= 0.20 else TERMINAL_COLORS["critical"]
            }
        else:
            charge_level, charge_percent = TeslaSwarmCoordinator.determine_charge_level(soc)

            return {
                "status": "NO_TRANSFER",
                "reasoning": f"SOC {soc:.2f} is above the {TeslaSwarmCoordinator.TRANSFER_THRESHOLD} threshold",
                "decision": "NO TRANSFER NEEDED",
                "details": f"Continue monitoring - {drone_name} has {charge_level} charge ({charge_percent})",
                "icon": "✅",
                "color": TERMINAL_COLORS["success"]
            }

# ============================================================================
# DISPLAY & OUTPUT FUNCTIONS
# ============================================================================

def print_system_header():
    """Print Tesla Swarm System header"""
    colors = TERMINAL_COLORS

    print(f"\n{colors['header']}{'='*80}{colors['reset']}")
    print(f"{colors['header']}           TESLA PERPETUAL SWARM ENERGY COORDINATOR           {colors['reset']}")
    print(f"{colors['header']}                    PRODUCTION SYSTEM v5.0                    {colors['reset']}")
    print(f"{colors['header']}           Model: meta/llama-3.1-405b-instruct               {colors['reset']}")
    print(f"{colors['header']}           Mission Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}{colors['reset']}")
    print(f"{colors['header']}{'='*80}{colors['reset']}\n")

def print_drone_report(drone_number, total_drones, drone_info, coordinator_output, llm_analysis=""):
    """Print formatted drone analysis report"""
    colors = TERMINAL_COLORS

    print(f"{colors['info']}⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯{colors['reset']}")
    print(f"{colors['info']}DRONE #{drone_number:02d}/{total_drones:02d}: {drone_info['name']}{colors['reset']}")
    print(f"{colors['info']}Status Report: SOC {drone_info['soc']} | Position: {drone_info['position']}{colors['reset']}")
    print(f"{colors['info']}⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯{colors['reset']}")

    # Coordinator decision
    print(f"\n{coordinator_output['color']}{coordinator_output['icon']} COORDINATOR DECISION:{colors['reset']}")
    print(f"{coordinator_output['color']}  • Reasoning: {coordinator_output['reasoning']}{colors['reset']}")
    print(f"{coordinator_output['color']}  • Decision:  {coordinator_output['decision']}{colors['reset']}")
    print(f"{coordinator_output['color']}  • Details:   {coordinator_output['details']}{colors['reset']}")

    # LLM analysis (if available)
    if llm_analysis and len(llm_analysis) > 10:
        analysis_preview = llm_analysis[:120].split('.')[0] + '.'
        print(f"\n{colors['info']}  📝 AI Analysis: {analysis_preview}{colors['reset']}")

    print(f"{colors['info']}⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯{colors['reset']}\n")

def print_mission_summary(mission_stats):
    """Print mission summary statistics"""
    colors = TERMINAL_COLORS

    print(f"\n{colors['header']}{'='*80}{colors['reset']}")
    print(f"{colors['header']}                    MISSION SUMMARY REPORT                    {colors['reset']}")
    print(f"{colors['header']}{'='*80}{colors['reset']}")

    # Statistics
    print(f"\n{colors['info']}📊 FLEET STATUS ANALYSIS:{colors['reset']}")
    print(f"   Total Drones Processed: {mission_stats['total']}")
    print(f"   {colors['warning']}⚡ Power Transfers Required: {mission_stats['transfers']}{colors['reset']}")
    print(f"   {colors['success']}✅ No Transfer Needed: {mission_stats['no_transfers']}{colors['reset']}")
    print(f"   {colors['info']}🔍 Manual Inspections Required: {mission_stats['inspections']}{colors['reset']}")

    # Priority breakdown
    if mission_stats['transfers'] > 0:
        print(f"\n{colors['warning']}⚡ TRANSFER PRIORITY BREAKDOWN:{colors['reset']}")
        print(f"   {colors['critical']}🔴 CRITICAL Priority: {mission_stats.get('critical', 0)} drone(s){colors['reset']}")
        print(f"   {colors['warning']}🟡 HIGH Priority: {mission_stats.get('high', 0)} drone(s){colors['reset']}")
        print(f"   {colors['success']}🟢 STANDARD Priority: {mission_stats.get('standard', 0)} drone(s){colors['reset']}")

    # Efficiency metrics
    if mission_stats['total'] > 0:
        efficiency = (mission_stats['no_transfers'] / mission_stats['total']) * 100
        print(f"\n{colors['success']}📈 FLEET EFFICIENCY METRICS:{colors['reset']}")
        print(f"   Operational Efficiency: {efficiency:.1f}%")

        # Efficiency rating
        if efficiency >= 75:
            rating = "⭐ EXCELLENT"
            rating_color = colors['success']
        elif efficiency >= 50:
            rating = "✓ GOOD"
            rating_color = colors['success']
        elif efficiency >= 25:
            rating = "⚠️  FAIR"
            rating_color = colors['warning']
        else:
            rating = "🔴 NEEDS ATTENTION"
            rating_color = colors['critical']

        print(f"   Fleet Rating: {rating_color}{rating}{colors['reset']}")

    # System status
    print(f"\n{colors['header']}⚙️  SYSTEM STATUS:{colors['reset']}")
    print(f"   Model: meta/llama-3.1-405b-instruct")
    print(f"   Business Rules: {colors['success']}ACTIVE{colors['reset']}")
    print(f"   Format Enforcement: {colors['success']}ENFORCED{colors['reset']}")
    print(f"   Output Consistency: {colors['success']}GUARANTEED{colors['reset']}")

    print(f"\n{colors['header']}{'='*80}{colors['reset']}")
    print(f"{colors['success']}✅ MISSION COMPLETE: All systems operational{colors['reset']}")
    print(f"{colors['header']}{'='*80}{colors['reset']}")

# ============================================================================
# MAIN PRODUCTION SYSTEM
# ============================================================================

async def run_production_swarm_system():
    """Main Tesla Swarm Production System"""

    # Save configuration
    config_path = "tesla_swarm_production_config.yml"
    with open(config_path, "w", encoding="utf-8") as f:
        yaml.dump(PRODUCTION_CONFIG, f, sort_keys=False, default_flow_style=False)

    print(f"{TERMINAL_COLORS['info']}Initializing Tesla Swarm Production System...{TERMINAL_COLORS['reset']}")
    print_system_header()

    # Define operational drone fleet
    operational_fleet = [
        {"name": "ALPHA-1", "soc": "SOC 0.12", "position": [45.5, -73.6, 10000]},
        {"name": "BETA-2", "soc": "SOC 0.92", "position": [45.7, -73.5, 9500]},
        {"name": "GAMMA-3", "soc": "SOC 0.84", "position": [45.6, -73.55, 10500]},
        {"name": "DELTA-4", "soc": "SOC: 0.85", "position": [45.55, -73.58, 9800]},
        {"name": "EPSILON-5", "soc": "SOC=0.99", "position": [45.65, -73.52, 10200]},
        {"name": "ZETA-6", "soc": "SOC status unknown", "position": [46.0, -74.0, 11000]},
        {"name": "THETA-7", "soc": "SOC error", "position": [45.8, -73.7, 9000]},
        {"name": "IOTA-8", "soc": "State of Charge 0.08", "position": [45.4, -73.65, 11500]},
        {"name": "KAPPA-9", "soc": "SOC 0.45", "position": [45.75, -73.8, 9200]},
        {"name": "LAMBDA-10", "soc": "SOC 0.78", "position": [45.3, -73.9, 10700]}
    ]

    # Initialize mission statistics
    mission_stats = {
        "total": 0,
        "transfers": 0,
        "no_transfers": 0,
        "inspections": 0,
        "critical": 0,
        "high": 0,
        "standard": 0
    }

    try:
        # Initialize workflow with 405B model
        async with load_workflow(config_path) as workflow:

            print(f"{TERMINAL_COLORS['info']}🚀 Processing {len(operational_fleet)} drones with 405B model...{TERMINAL_COLORS['reset']}\n")

            # Process each drone in the fleet
            for idx, drone in enumerate(operational_fleet, 1):
                mission_stats["total"] += 1

                # Create task for LLM
                task = f"{drone['name']} reports {drone['soc']} at position {drone['position']}"

                # Get AI analysis from 405B model
                async with workflow.run(task) as runner:
                    llm_analysis = await runner.result()

                # Extract SOC and analyze
                soc = TeslaSwarmCoordinator.extract_soc_from_report(task)
                is_faulty = TeslaSwarmCoordinator.is_faulty_report(drone['soc'])

                # Generate coordinator decision
                coordinator_output = TeslaSwarmCoordinator.generate_coordinator_output(
                    drone['name'], soc, drone['position'], is_faulty
                )

                # Update mission statistics
                if coordinator_output["status"] == "TRANSFER_REQUIRED":
                    mission_stats["transfers"] += 1
                    if soc is not None:
                        if soc < 0.20:
                            mission_stats["critical"] += 1
                        elif soc < 0.50:
                            mission_stats["high"] += 1
                        else:
                            mission_stats["standard"] += 1
                elif coordinator_output["status"] == "NO_TRANSFER":
                    mission_stats["no_transfers"] += 1
                else:
                    mission_stats["inspections"] += 1

                # Print drone report
                print_drone_report(
                    idx, len(operational_fleet),
                    drone, coordinator_output, llm_analysis
                )

            # Print mission summary
            print_mission_summary(mission_stats)

            # Save operational report
            print(f"\n{TERMINAL_COLORS['success']}💾 Configuration saved to: {config_path}{TERMINAL_COLORS['reset']}")
            print(f"{TERMINAL_COLORS['success']}🚀 Production System: OPERATIONAL AND READY{TERMINAL_COLORS['reset']}")

    except Exception as e:
        print(f"\n{TERMINAL_COLORS['critical']}❌ SYSTEM ERROR: {str(e)}{TERMINAL_COLORS['reset']}")
        print(f"{TERMINAL_COLORS['warning']}Please verify: API Key | Model Access | Network Connection{TERMINAL_COLORS['reset']}")

# ============================================================================
# EXECUTE PRODUCTION SYSTEM
# ============================================================================

# Run the production system
print(f"{TERMINAL_COLORS['info']}Starting Tesla Perpetual Swarm Energy Coordinator...{TERMINAL_COLORS['reset']}")
await run_production_swarm_system()

# Additional utility functions
def get_system_info():
    """Get system information"""
    return {
        "system": "Tesla Perpetual Swarm Energy Coordinator",
        "version": "5.0",
        "model": "meta/llama-3.1-405b-instruct",
        "business_rule": "SOC ≤ 0.85 → Power Transfer",
        "output_format": "Reasoning, Decision, Details",
        "status": "Production Ready"
    }

print(f"\n{TERMINAL_COLORS['info']}System Information:{TERMINAL_COLORS['reset']}")
for key, value in get_system_info().items():
    print(f"  {key.replace('_', ' ').title()}: {TERMINAL_COLORS['success']}{value}{TERMINAL_COLORS['reset']}")

Starting Tesla Perpetual Swarm Energy Coordinator...
Initializing Tesla Swarm Production System...

           TESLA PERPETUAL SWARM ENERGY COORDINATOR           
                    PRODUCTION SYSTEM v5.0                    
           Model: meta/llama-3.1-405b-instruct               
           Mission Time: 2026-01-19 13:57:26

🚀 Processing 10 drones with 405B model...

⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
DRONE #01/10: ALPHA-1
Status Report: SOC SOC 0.12 | Position: [45.5, -73.6, 10000]
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯

⚡ COORDINATOR DECISION:
  • Reasoning: SOC 0.12 is below the 0.85 threshold
  • Decision:  TRANSFER REQUIRED
  • Details:   Initiate 🔴 CRITICAL priority power transfer to ALPHA-1. Immediate emergency transfer required

  📝 AI Analysis: It appears we have a situation report from ALPHA-1.
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯

⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
DRONE #02/10: BETA-2
Status Re